### Confirm we can use a GPU to run the model

In [1]:
import tensorflow as tf

In [2]:
# turn on memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
gpus = tf.config.list_physical_devices('GPU')
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")

1 Physical GPUs, 1 Logical GPU


In [4]:
from tensorflow.python.client import device_lib

In [5]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12491230001603738054,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 188612608
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12886346244392812598
 physical_device_desc: "device: 0, name: NVIDIA GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"]

In [6]:
from keras import backend as K

In [7]:
K.clear_session()

In [8]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [9]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
!nvidia-smi

Fri Jun 25 16:03:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   41C    P0    N/A /  N/A |   1606MiB /  2004MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Define constants

In [14]:
fn = "data/kuze_data/evaluations_per_ans_with_taxonomy_ids_PPL.csv"
verbose = 1
best_model_weights = "weights/bestmodel"
log_dir = "logs"
optimizer = "adam"
lstm_units = 50
batch_size = 32
epochs = 5
dropout_rate = 0.3
test_fraction = 0.2
validation_fraction = 0.2

### Pre-processing

In [15]:
import sys

sys.path.append('/home/grenouille/Documents/jenga/final_project/code/kuze_dkt_imp')

In [ ]:
from deepkt import deepkt, data_util, metrics

dataset, length, nb_features, nb_taxonomies = data_util.load_dataset(fn=fn, batch_size=batch_size, shuffle=True)

train_set, test_set, val_set = data_util.split_dataset(dataset=dataset, total_size=length, test_fraction=test_fraction, val_fraction=validation_fraction)

set_size = length * batch_size

test_set_size = (set_size * test_fraction)

val_set_size = (set_size - test_set_size) * validation_fraction

train_set_size = set_size - test_set_size - val_set_size

print("============== Data Summary ==============")
print("Total number of students: %d" % set_size)
print("Training set size: %d" % train_set_size)
print("Validation set size: %d" % val_set_size)
print("Testing set size: %d" % test_set_size)
print("Number of skills: %d" % nb_taxonomies)
print("Number of features in the input: %d" % nb_features)
print("========================================= ")

/home/grenouille/anaconda3/envs/tf_gpu/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (15,18,19,29,30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


> /home/grenouille/Documents/jenga/final_project/code/kuze_dkt_imp/deepkt/data_util.py(47)load_dataset()
     45     # Create a series for each student followed by a tuple of arrays for their
     46     # taxonomy_with_answer, factorized_taxonomy_code, answer_selection_correct
---> 47     seq = df.groupby('student_id').apply(
     48         lambda r: (
     49             r['taxonomy_with_answer'].values[:-1],



### Building the model

In [14]:
student_model = deepkt.DKTModel(
        nb_features=nb_features,
        nb_taxonomies=nb_taxonomies,
        hidden_units=lstm_units,
        dropout_rate=dropout_rate)

student_model.compile(
        optimizer=optimizer,
        metrics=[
            metrics.BinaryAccuracy(),
            metrics.AUC(),
            metrics.Precision(),
            metrics.Recall()
        ])

student_model.summary()

Model: "DKTModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, None, 700)]       0         
_________________________________________________________________
masking (Masking)            (None, None, 700)         0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 50)          150200    
_________________________________________________________________
outputs (TimeDistributed)    (None, None, 350)         17850     
Total params: 168,050
Trainable params: 168,050
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [15]:
import tensorflow as tf
history = student_model.fit(
    dataset=train_set,
    epochs=epochs,
    verbose=verbose,
    validation_data=val_set,
    callbacks=[
        tf.keras.callbacks.CSVLogger(f"{log_dir}/train.log"),
        tf.keras.callbacks.ModelCheckpoint(best_model_weights, save_best_only=True, save_weights_only=True),
        tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    ]
)

Epoch 1/5
      5/Unknown - 15s 1s/step - loss: 0.0684 - binary_accuracy: 0.5196 - auc: 0.5162 - precision: 0.4683 - recall: 0.4867

CancelledError:  [_Derived_]RecvAsync is cancelled.
	 [[{{node broadcast_weights_1/assert_broadcastable/is_valid_shape/else/_1/broadcast_weights_1/assert_broadcastable/is_valid_shape/has_valid_nonscalar_shape/then/_392/broadcast_weights_1/assert_broadcastable/is_valid_shape/has_valid_nonscalar_shape/has_invalid_dims/concat/_30}}]] [Op:__inference_train_function_8166]

Function call stack:
train_function


### Load the model with the best validation loss

In [ ]:
student_model.load_weights(best_model_weights)

### Test the model

In [ ]:
result = student_model.evaluate(test_set, verbose=verbose)

In [ ]:
result